<a href="https://colab.research.google.com/github/edki96/Kio-Mugweru-Projects/blob/main/Credit_and_Collections_Weekend_Duty_2025_2026.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import random

# Setup team and Kenyan public holidays
team = [f"Staff {i+1}" for i in range(20)]
public_holidays_str = [
    "2025-01-01", "2025-03-31", "2025-04-18", "2025-04-21", "2025-05-01",
    "2025-06-01", "2025-06-07", "2025-10-10", "2025-10-20", "2025-12-12",
    "2025-12-25", "2025-12-26"
]
public_holidays = set(datetime.strptime(d, "%Y-%m-%d").date() for d in public_holidays_str)

# Generate all Saturdays, Sundays, and public holidays in 2025
start_date = datetime(2025, 6, 14).date()
end_date = datetime(2026, 3, 31).date()

all_duty_dates = []
cur_date = start_date
while cur_date <= end_date:
    if cur_date.weekday() in (5, 6) or cur_date in public_holidays:
        all_duty_dates.append(cur_date)
    cur_date += timedelta(days=1)

def day_type(date):
    if date in public_holidays:
        return "Public Holiday"
    elif date.weekday() == 5:
        return "Saturday"
    else:
        return "Sunday"

# Tracking assignments
staff_total_assignments = {s: 0 for s in team}
staff_month_assignments = {s: set() for s in team}

# Rotation pool to ensure everyone works once before repeats
rotation_pool = team.copy()
random.shuffle(rotation_pool)

assignments = []

for date in sorted(all_duty_dates):
    d_type = day_type(date)
    needed = 2 if d_type in ("Saturday", "Public Holiday") else 1

    # Eligible staff: in rotation pool and not worked this month
    month = date.month
    eligible = [s for s in rotation_pool if month not in staff_month_assignments[s]]

    # If not enough eligible, reset rotation pool and reshuffle
    if len(eligible) < needed:
        rotation_pool = team.copy()
        random.shuffle(rotation_pool)
        eligible = [s for s in rotation_pool if month not in staff_month_assignments[s]]

    # Sort eligible by total assignments to balance workload
    eligible.sort(key=lambda s: staff_total_assignments[s])

    # Randomize the order for fairness
    random.shuffle(eligible)

    assigned = eligible[:needed]

    for s in assigned:
        staff_total_assignments[s] += 1
        staff_month_assignments[s].add(month)
        if s in rotation_pool:
            rotation_pool.remove(s)

    # Pad assigned list for consistent 2 columns output
    while len(assigned) < 2:
        assigned.append("")

    assignments.append([date.strftime("%Y-%m-%d"), d_type, assigned[0], assigned[1]])

# Export to Excel
df = pd.DataFrame(assignments, columns=["Date", "Day Type", "Staff 1", "Staff 2"])
df.to_excel("Credit_and_Collections_Weekend_Duty_2025.xlsx", index=False)

print("Schedule saved to 'Credit_and_Collections_Weekend_Duty_2025.xlsx'")


Schedule saved to 'Credit_and_Collections_Weekend_Duty_2025.xlsx'
